# Read data

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Tokenize

### Pre process

In [3]:
import re

#DEMO
text = "Hello, world. This, is a test."
result = re.split(r'([,.?_!"()\']|--|\s)', text)
# print(result)

#remove white spaces
result = [item for item in result if item.strip()]
# print(result)

# TEXT:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4649
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Create Vocabulary

In [4]:

all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(f'{vocab_size=}')

vocab = {token:integer for integer ,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

vocab_size=1159
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)
('Her', 51)


In [5]:
from tokenizer import SimpleTokenizerV1

tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]
"  It'  s  the  last  he  painted,  you  know,"  Mrs.  Gisburn  said  with  pardonable  pride.


### Adding < unk > and <|endoftext|>

In [6]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab.items()))

1161


In [7]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [8]:
from tokenizer import SimpleTokenizerV2

tokenizer = SimpleTokenizerV2(vocab)
ids =tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]
<|unk|>,  do  you  like  tea?  <|endoftext|>  In  the  sunlit  terraces  of  the  <|unk|>.


## Byte Pair encoding

In [9]:
from importlib.metadata import version

import tiktoken 
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.7.0


In [10]:
tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
print(f"Decode: {tokenizer.decode(integers)}")

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
Decode: Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


### Encode dataset

In [11]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [12]:
enc_sample = enc_text[50:]

context_size = 4 #A
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x} : {tokenizer.decode(x)}")
print(f"y: {y}: {tokenizer.decode(y)}")

x: [290, 4920, 2241, 287] :  and established himself in
y: [4920, 2241, 287, 257]:  established himself in a


In [13]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)
    print(context, "---->", desired)

[290] ----> 4920
[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
[290, 4920, 2241, 287] ----> 257


### dataloader

In [14]:
from dataloader import GPTDatasetV1
from torch.utils.data import DataLoader

def create_dataloader_v1(txt, batch_size=4,
    max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2") #A
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) #B
    dataloader = DataLoader(
                    dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last) #C
    return dataloader


dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride = 4, shuffle=False)
data_iter = iter(dataloader)


In [15]:

first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


## Embedding 

In [16]:
import torch

vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [17]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## Positional Embedding

In [21]:
# Step 1: Embedding layer
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(f'{token_embedding_layer.weight.shape=}')

token_embedding_layer.weight.shape=torch.Size([50257, 256])


In [27]:
# Step 2: Data set with dataloader.

max_length = 4
dataloader = create_dataloader_v1(
                raw_text, batch_size=8, max_length=max_length, 
                stride=max_length, shuffle=True)


Token IDs:
 tensor([[  561,   423,   587, 10598],
        [  338, 10568,   550,   587],
        [ 1544,  9373,  6364,    25],
        [ 1337,   257, 14787,   618],
        [28060,     6,   416,   617],
        [ 3499,   438,   361,   314],
        [   88, 22870,   286,  9568],
        [ 7428,   262,  3394, 20096]])

Inputs shape:
 torch.Size([8, 4])


In [30]:
# Step 3: Get token embeddings for input:

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

token_embeddings = token_embedding_layer(inputs)
print(f"{token_embeddings=}")

Token IDs:
 tensor([[ 4119,    81,   621,   339],
        [  510,   379,   262, 17548],
        [  314,   836,   470, 45553],
        [  587,  6776,    13,   314],
        [  340,   438,    62,    40],
        [   13,   198,   198,  3347],
        [   13,  8759,  2763,   438],
        [  340,  1422,   470,  1011]])

Inputs shape:
 torch.Size([8, 4])
token_embeddings=tensor([[[ 1.3138, -0.5494, -0.5770,  ..., -0.0681, -0.3923, -1.9574],
         [ 1.1834, -1.2588,  0.5658,  ..., -0.2538,  0.0773, -0.1936],
         [ 0.6738,  0.6684,  1.3194,  ..., -0.0778,  0.1999, -0.4613],
         [-0.8439,  1.1579,  0.0071,  ...,  0.7550,  0.0506, -0.0256]],

        [[ 0.2329,  0.8564,  0.9211,  ...,  0.7988,  0.4715,  0.3352],
         [ 0.7873, -0.0319, -0.7761,  ...,  0.0928, -1.2866, -1.0368],
         [-0.4499, -0.8161, -0.8413,  ..., -0.4998, -1.1656, -1.4683],
         [-0.8463,  1.5084, -1.6118,  ..., -0.0201, -0.0150,  0.4078]],

        [[ 0.6098, -0.5960,  0.8533,  ...,  0.6239,  0.7711

In [31]:
# step 4: Position embedding layer.

context_lenght = max_length
pos_embedding_layer = torch.nn.Embedding(context_lenght, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding_layer.weight.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 0.1014, -0.4700, -1.2878,  ..., -0.5435, -0.6292,  1.2414],
        [-0.8162, -1.3853, -0.3122,  ..., -1.2092, -0.1453,  0.4243],
        [-0.6036,  1.2745,  1.1917,  ...,  0.4839, -0.6934, -0.2210],
        [ 0.3730, -0.9644, -1.3848,  ..., -1.6185,  0.3091, -0.2043]],
       grad_fn=<EmbeddingBackward0>)


In [33]:
# Step 4: Final input embeddings

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 1.4152, -1.0194, -1.8648,  ..., -0.6115, -1.0215, -0.7160],
         [ 0.3673, -2.6441,  0.2536,  ..., -1.4630, -0.0679,  0.2307],
         [ 0.0702,  1.9429,  2.5111,  ...,  0.4060, -0.4934, -0.6823],
         [-0.4708,  0.1936, -1.3777,  ..., -0.8635,  0.3597, -0.2298]],

        [[ 0.3343,  0.3864, -0.3667,  ...,  0.2553, -0.1576,  1.5766],
         [-0.0288, -1.4172, -1.0883,  ..., -1.1164, -1.4319, -0.6125],
         [-1.0535,  0.4584,  0.3504,  ..., -0.0160, -1.8589, -1.6893],
         [-0.4732,  0.5441, -2.9967,  ..., -1.6385,  0.2941,  0.2035]],

        [[ 0.7112, -1.0660, -0.4344,  ...,  0.0804,  0.1419, -0.6029],
         [-2.3781, -0.5745, -0.2693,  ..., -0.9307,  0.8405,  1.4878],
         [-0.0665,  0.3427,  0.1017,  ...,  1.7246, -0.7879, -0.3355],
         [ 0.2895, -0.7402, -1.5526,  ..., -1.2547, -1.5433,  0.4152]],

        ...,

        [[-0.0180, -0.0515, -0.2313,  ...,  0.0816,  0.6511,  3.5366],
         [ 1.0951, -0.9153, -1.09